# Solving the Hamiltonian in different Ways



## Generating an XXZ Hamiltonian

### What is the XXZ Hamiltonian?

We first must generate some XXZ Hamiltonian. This describes the total energy of all of the 'spin-arrows'. 

- The XX part describes the tendency of neighbouring spins to align in the horizontal (x) and vertical (y) directions. This interaction is isotropic.
- THE Z part describes the interaction strength between spins in the "up/down" (z) direction.

The whole Hamiltonian is

$$H = J \sum_{i=1}^{L-1} (S_i^xS^x_{i+1} + S^y_iS^y_{i+1} + \Delta S^z_i S^z_{i+1})$$

- $J$ is the **coupling constant**. If $J>0$ then neighbouring spins like to point in opposite directions (antiferromagnetic). If $J<0$ then neighbouring spins like to point in the same direction (ferromagnetic). 
- The sum means that we sum over all adjacent pairs of sites, from site 1 and 2, up to site $L-1$ and $L$. This describes an "open chain". 
- $S^x_i$ is the component of spin in the $x$-direction at site $i$, and so on.
- $\Delta$ is the **anisotropy parameter**, which tunes the strength of the $z$ interaction.

For a spin-1/2 system, the spin operators are represented by the Pauli matrices i.e.

$$S^x = \frac{1}{2} \begin{pmatrix}0 & 1 \\ 1 & 0 \end{pmatrix}, \quad S^y = \frac{1}{2} \begin{pmatrix}0 & -i \\ i & 0 \end{pmatrix}, \quad S^z = \frac{1}{2} \begin{pmatrix}1 & 0 \\ 0 & 1 \end{pmatrix}$$

### Building the full Hamiltonian

The real challenge is constructing the Hamiltonian for $L$ spins - the total system size is $2^L \times 2^L$ which grows fast with $L$. 

To get an operator that acts only on site $i$, we need the tensor product. For example, the operator $S^z_i$ for a 3-spin ($L=3$) chain is 

$$S^z_1 = S^z \otimes I \otimes I, \quad S^z_2 = I \otimes S^z \otimes I, \quad S^z_3 = I \otimes I \otimes S^z \\$$

In [8]:
using LinearAlgebra

function get_xxz_hamiltonian(L::Int, J::Float64, Δ::Float64)
    """
    Constructs the Hamiltonian for the XXZ spin chain model with L sites, coupling constant J, and anisotropy parameter Δ.
    """
    dim = 2^L

    # define the pauli matrices we need
    sx = [0.0 1.0; 1.0 0.0] # pauli X
    sy = [0.0 -im; im 0.0] # pauli Y
    sz = [1.0 0.0; 0.0 -1.0] # pauli Z
    id = I(2) # 2D identity from LinearAlgebra

    # init as a zero matrix
    H = zeros(ComplexF64, dim, dim)

    # now we can construct the hamiltonian (i.e. fill it with the right values)
    # loop over all the sites 0 to L-1
    for i in 1:L-1
        j = i + 1 # j is the adjacent site to i (to the right)

        # we need to construct the full terms for each interaction
        term_xx = 1.0
        term_yy = 1.0
        term_zz = 1.0

        for site in 1:L
            if site == i
                # if we are on the ith site, we want to add the sx, sy, sz terms
                term_xx = kron(term_xx, sx)
                term_yy = kron(term_yy, sy)
                term_zz = kron(term_zz, sz)
            elseif site == j 
                # if we are on the jth site, we want to add the sx, sy, sz terms
                term_xx = kron(term_xx, sx)
                term_yy = kron(term_yy, sy)
                term_zz = kron(term_zz, sz)
            else
                # otherwise we add the identity
                term_xx = kron(term_xx, id)
                term_yy = kron(term_yy, id)
                term_zz = kron(term_zz, id)
            end
        end

        # now we can add these terms to the hamiltonian
        H += J * term_xx + J * term_yy + Δ * term_zz
    end

    return H
end


get_xxz_hamiltonian (generic function with 1 method)

#### Note - what does `kron` do?
`kron(a,b)` computes the Kroenecker product of two vectors/matrices/numbers. 

In [17]:

test_hamiltonian = get_xxz_hamiltonian(4, 1.0, 1.5)
display(real(test_hamiltonian))
eigens = eigen(test_hamiltonian)
eigenvalues = eigens.values
display(eigenvalues)
ground_state = minimum(eigenvalues)
println("Ground state energy: $ground_state\n")

16×16 Matrix{Float64}:
 4.5  0.0   0.0  0.0   0.0   0.0   0.0  …   0.0   0.0  0.0   0.0  0.0  0.0
 0.0  1.5   2.0  0.0   0.0   0.0   0.0      0.0   0.0  0.0   0.0  0.0  0.0
 0.0  2.0  -1.5  0.0   2.0   0.0   0.0      0.0   0.0  0.0   0.0  0.0  0.0
 0.0  0.0   0.0  1.5   0.0   2.0   0.0      0.0   0.0  0.0   0.0  0.0  0.0
 0.0  0.0   2.0  0.0  -1.5   0.0   0.0      0.0   0.0  0.0   0.0  0.0  0.0
 0.0  0.0   0.0  2.0   0.0  -4.5   2.0  …   0.0   0.0  0.0   0.0  0.0  0.0
 0.0  0.0   0.0  0.0   0.0   2.0  -1.5      2.0   0.0  0.0   0.0  0.0  0.0
 0.0  0.0   0.0  0.0   0.0   0.0   0.0      0.0   2.0  0.0   0.0  0.0  0.0
 0.0  0.0   0.0  0.0   2.0   0.0   0.0      0.0   0.0  0.0   0.0  0.0  0.0
 0.0  0.0   0.0  0.0   0.0   2.0   0.0      2.0   0.0  0.0   0.0  0.0  0.0
 0.0  0.0   0.0  0.0   0.0   0.0   2.0  …  -4.5   0.0  2.0   0.0  0.0  0.0
 0.0  0.0   0.0  0.0   0.0   0.0   0.0      0.0  -1.5  0.0   2.0  0.0  0.0
 0.0  0.0   0.0  0.0   0.0   0.0   0.0      2.0   0.0  1.5   0.0  0.0  0.0
 0

16-element Vector{Float64}:
 -7.574673582515121
 -5.105551275463986
 -4.201562118716412
 -4.2015621187164065
 -1.4999999999999916
 -1.0615528128088285
 -1.0615528128088196
  0.38679098625037467
  2.1055512754639887
  2.201562118716424
  2.201562118716426
  2.687882596264753
  3.0615528128088303
  3.0615528128088303
  4.5
  4.5

Ground state energy: -7.574673582515121



### Issues with this

This is very slow... For $L=100$, the number of states is larger than the number of atoms in the universe, which cannot be stored on any computer.

## Density Matrix Renormalization Group (DMRG)

The core idea behind DMRG is to focus only on what's important, and forget everything else. But what does imporatant actually mean?

We use a **density matrix**, which tells us how strongly one part of the system is entangled with another. By analysing the density matrix, DMRG can make a very intelligent decision about which states to discard. 

### How it works

DMRG builds a spin chain iteratively, growing the system and refining its description at each step. 

In [60]:
# first we need a function to generate a MPS representation of the ground state
using ITensors 
using ITensorMPS

function create_MPS(L::Int, Χ::Int)
    """Create a random MPS for a spin-1/2 chain of length L with bond dimension Χ."""
    # create a site set for a spin-1/2 chain
    sites = siteinds("S=1/2", L; conserve_qns=true) # conserve total Sz

    # create a random MPS with bond dimension Χ
    state = [isodd(i) ? "Up" : "Dn" for i in 1:L] # generic ground state
    ψ0 = randomMPS(sites, state, Χ)
    return ψ0, sites
end

create_MPS (generic function with 1 method)

In [64]:
function get_xxz_hamiltonian_itensor(sites, J::Float64, Δ::Float64)
    """
    Constructs the Hamiltonian for the XXZ spin chain model using ITensors.jl with given sites, coupling constant J, and anisotropy parameter Δ.
    """
    L = length(sites)
    ampo = AutoMPO()

    for i in 1:L-1
        j = i + 1
        add!(ampo, J, "Sx", i, "Sx", j)
        add!(ampo, J, "Sy", i, "Sy", j)
        add!(ampo, Δ, "Sz", i, "Sz", j)
    end

    H = MPO(ampo, sites)
    return H
end

get_xxz_hamiltonian_itensor (generic function with 1 method)

In [ ]:
# parameters
L = 4 # length of the spin chain
Χ = 10 # bond dimension
J = 1.0 # coupling constant
Δ = 1.5 # anisotropy parameter

# DMRG parameters
sweeps = Sweeps(15)
setmaxdim!(sweeps,χ)
setcutoff!(sweeps, 1E-14)

ψ0, sites = create_MPS(L, Χ)
H = get_xxz_hamiltonian_itensor(sites, J, Δ)
E, ψ = dmrg(H, ψ0; sweeps=sweeps)
println("Ground state energy from DMRG: $E")

┌ Warning: MPS center bond dimension is less than requested (you requested 10, but in practice it is 4. This is likely due to technicalities of truncating quantum number sectors.
└ @ ITensorMPS /Users/jamesneville-rolfe/.julia/packages/ITensorMPS/AQY99/src/mps.jl:153


ErrorException: Fluxes not all equal

In [59]:
using Plots
using ITensors

function plot_magnetization_manual(ψ, sites)
    L = length(sites)
    mag_z = Float64[]
    
    for i in 1:L
        # Method 1: Create Sz operator and compute <ψ|Sz_i|ψ>
        orthogonalize!(ψ, i)  # Orthogonalize MPS at site i
        sz_op = op("Sz", sites[i])  # Create Sz operator for site i
        mag_i = real(scalar(dag(ψ[i]) * sz_op * ψ[i]))
        push!(mag_z, mag_i)
    end
    
    plot(1:L, mag_z, 
         marker=:circle, 
         title="Local Magnetization ⟨Sz⟩ (Manual)",
         xlabel="Site", 
         ylabel="⟨Sz⟩",
         legend=false)
end

plot_magnetization_manual(ψ0, sites)

ErrorException: Attempting to contract IndexSet:

((dim=2|id=150|"S=1/2,Site,n=1") <In>
 1: QN("Sz",1) => 1
 2: QN("Sz",-1) => 1, (dim=2|id=889|"Link,l=1") <In>
 1: QN("Sz",1) => 1
 2: QN("Sz",-1) => 1)

with IndexSet:

((dim=2|id=150|"S=1/2,Site,n=1")' <Out>
 1: QN("Sz",1) => 1
 2: QN("Sz",-1) => 1, (dim=2|id=150|"S=1/2,Site,n=1") <In>
 1: QN("Sz",1) => 1
 2: QN("Sz",-1) => 1)

QN indices must have opposite direction to contract, but indices:

(dim=2|id=150|"S=1/2,Site,n=1") <In>
 1: QN("Sz",1) => 1
 2: QN("Sz",-1) => 1

and:

(dim=2|id=150|"S=1/2,Site,n=1") <In>
 1: QN("Sz",1) => 1
 2: QN("Sz",-1) => 1

do not have opposite directions.